<a href="https://colab.research.google.com/github/kato-taki/optimization-of-cultured-meat/blob/main/%E2%98%851101_optimization_lambda_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Input

## Import library


In [ ]:
#optunaのインストール
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 6.1 MB/s eta 0:00:00


In [ ]:
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
import math
import time
from scipy.optimize import curve_fit
from scipy.optimize import minimize
import pandas as pd
import random
import statistics
import csv
import optuna

## Parameter value and Constant value, experimental condition


In [ ]:
# すべてのパラメータをリスト形式で定義
data = [
    #===========================================================================
    # 動物細胞培養工程モジュール (C2C12マウス筋芽細胞)
    #===========================================================================
    # モデルパラメータの初期値および推定値
    # 播種密度
    ["X0_cell",     3.664e7],
    # 培地濃度
    ["Cglc0_cell",  23.2],
    ["Clac0_cell",  8.5887],
    ["Cgln0_cell",  1.4236],
    ["Camm0_cell",  0.83965],
    # 最大比増殖速度
    ["umax_cell",   0.04280950136113015],
    #最小比増殖速度
    ["umin_cell",   1e-2],
    # q = ΔC/ΔX (mmol/cells)
    ["qglc_cell_1", 2.67436e-7],  #1サイクル目
    ["qglc_cell_2", 2.02496e-7],  #2サイクル目以降
    ["qlac_cell_1", 3.95337e-7],  #1サイクル目
    ["qlac_cell_2", 2.99975e-7],  #2サイクル目以降
    ["qgln_cell",   3.898774029384385e-09],
    ["qamm_cell",   1.533722182522993e-09],

    # 動物細胞の変動係数 (CV)
    # 播種密度
    ["CV_X0_cell",  8.648e-2],
    # カウント誤差
    ["CV_X_count",    0.04],
    # 培地濃度
    ["CV_Cglc0_cell", 1.701e-2],
    ["CV_Clac0_cell", 3.366e-2],
    ["CV_Cgln0_cell", 6.962e-2],
    ["CV_Camm0_cell", 0.1042],
    # 最大比増殖速度
    ["CV_umax_cell", 8.104e-2],
    # q = ΔC/ΔX (mmol/cells)
    ["CV_qglc_cell", 8.041e-2],
    ["CV_qlac_cell", 6.476e-2],
    ["CV_qgln_cell", 9.956e-2],
    ["CV_qamm_cell", 0.1323],


    #===========================================================================
    # 藻類培養工程モジュール (synechococcus sp.)
    #===========================================================================
    # モデルパラメータの初期値および推定値
    # 播種密度
    ["X0_algae", 4.667e10],
    #培地濃度
    ["Cglc0_algae", 18.6505],
    ["Clac0_algae", 34.70],
    ["Cgln0_algae", 1.4236],
    ["Camm0_algae", 2.94],
    # 最大比増殖速度
    ["umax_algae", 0.002753245657083259],
    # q = ΔC/ΔX (mmol/cells)
    ["qglc_algae", 1.21806e-11],
    ["qlac_algae", 1.85181e-9],
    ["qgln_algae", -4.194886042546756e-12],
    ["qamm_algae", 1.3534202942687656e-10],

    # 藻類の変動係数 (CV)
    # 播種密度
    ["CV_X0_algae", 1.101e-2],
    # 培地濃度
    ["CV_Cglc0_algae", 0.2072],
    ["CV_Clac0_algae", 4.389e-3],
    ["CV_Cgln0_algae", 0.1282],
    ["CV_Camm0_algae", 8.478e-2],
    # 最大比増殖速度
    ["CV_umax_algae", 0.1597],
    # # q = ΔC/ΔX (mmol/cells)
    ["CV_qglc_algae", 2.4162],
    ["CV_qlac_algae", 0.2372],
    ["CV_qgln_algae", 1.192],
    ["CV_qamm_algae", 0.1778],


    #===========================================================================
    # 藻類栄養素抽出工程モジュール
    #===========================================================================
    # C2C12細胞・乳酸資化シネココッカス循環培養より推定
    ["kglc", 2.13e-9],    #7.7075099320995e-10
    ["CV_kglc", 0.207],  #0.0993444775580169


    #===========================================================================
    # 培養操作パラメータ
    #===========================================================================
    # 抽出した栄養素を藻類廃培地に添加する割合
    ["k_hydrolysis", 5/100],
    # 動物細胞培養タンクの容積
    ["V", 1],


    #===========================================================================
    # 定数および実験条件
    #===========================================================================
    # 飽和定数KS
    ["Kglc", 1.7798238103747508],
    ["Kgln", 0.1],
    ["Kamm_algae", 0.03569506712587975],

    # 阻害定数KI
    ["Klac_cell", 50.15303470159003],
    ["Kamm_cell", 8.0],

    # グルタミン分解定数
    ["kd_gln_37", 3.6505e-3],
    ["kd_gln_25", 1.2038e-3],

    # 実験条件
    # トータルの培養期間
    ["t_tot", 480]
]

# グローバル変数に各パラメータを設定
for item in data:
    globals()[item[0]] = item[1]

# CSVファイルに書き込む
output_file = 'model_parameters_with_sd_cv.csv'
with open(output_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["Parameter", "Value"])  # ヘッダー行の書き込み
    writer.writerows(data)  # パラメータと値を行として書き込む

print(f"Model parameters have been written to {output_file}")

Model parameters have been written to model_parameters_with_sd_cv.csv


# Process

## Definition of culture process function

In [ ]:
# dt を適切に定義
dt = 0.05  # タイムステップの値

#===============================================================================
#動物細胞培養工程モジュール
#===============================================================================
#1サイクル目
def cell_culture_1(t, y):
    X,Cglc,Clac,Cgln,Camm = y
    #比増殖速度
    u = umax_cell *(Cglc/((Kglc+Cglc)*(1+Clac/Klac_cell)))*(Cgln/((Kgln+Cgln)*(1+Camm/Kamm_cell)))
    #増殖速度式
    dXdt = u * X

    #代謝速度式
    #グルコース代謝
    dCglcdt = -qglc_cell_1 * u * X
    # Cglcが負の値をとらないようにする制約
    if Cglc + dCglcdt * dt < 0:   # ここで dt は時間の微小な変化量（タイムステップ）を表す
      dCglcdt = -Cglc / dt        # dt で割ることでタイムステップに基づいて調整

    #乳酸代謝
    dClacdt = qlac_cell_1 * u * X

    #グルタミン代謝
    dCglndt = -qgln_cell * u * X - kd_gln_37 * Cgln
    # Cglnが負の値をとらないようにする制約
    if Cgln + dCglndt * dt < 0:   # ここで dt は時間の微小な変化量（タイムステップ）を表す
      dCglndt = -Cgln / dt        # dt で割ることでタイムステップに基づいて調整

    #アンモニア代謝
    dCammdt = qamm_cell * u * X + kd_gln_37 * Cgln

    return [dXdt, dCglcdt, dClacdt, dCglndt, dCammdt]

#2サイクル目以降
def cell_culture_2(t, y):
    X,Cglc,Clac,Cgln,Camm = y
    #比増殖速度
    u = umax_cell *(Cglc/((Kglc+Cglc)*(1+Clac/Klac_cell)))*(Cgln/((Kgln+Cgln)*(1+Camm/Kamm_cell)))
    #増殖速度式
    dXdt = u * X

    #代謝速度式
    #グルコース代謝
    dCglcdt = -qglc_cell_2 * u * X
    # Cglcが負の値をとらないようにする制約
    if Cglc + dCglcdt * dt < 0:   # ここで dt は時間の微小な変化量（タイムステップ）を表す
      dCglcdt = -Cglc / dt        # dt で割ることでタイムステップに基づいて調整

    #乳酸代謝
    dClacdt = qlac_cell_2 * u * X

    #グルタミン代謝
    dCglndt = -qgln_cell * u * X - kd_gln_37 * Cgln
    # Cglnが負の値をとらないようにする制約
    if Cgln + dCglndt * dt < 0:   # ここで dt は時間の微小な変化量（タイムステップ）を表す
      dCglndt = -Cgln / dt        # dt で割ることでタイムステップに基づいて調整

    #アンモニア代謝
    dCammdt = qamm_cell * u * X + kd_gln_37 * Cgln

    return [dXdt, dCglcdt, dClacdt, dCglndt, dCammdt]


#動物細胞の比増殖速度
def u_cell(Cglc, Clac, Cgln, Camm):
  u = umax_cell *(Cglc/((Kglc+Cglc)*(1+Clac/Klac_cell)))*(Cgln/((Kgln+Cgln)*(1+Camm/Kamm_cell)))
  return u

#===============================================================================
#藻類培養工程モジュール
#===============================================================================
def algae_culture(t, y):
  X,Cglc,Clac,Cgln,Camm = y
  #比増殖速度
  u = umax_algae *(Camm/(Kamm_algae+Camm))
  #増殖速度式
  dXdt = u * X

  #代謝速度式
  #グルコース代謝
  dCglcdt = -qglc_algae * u * X
  # Cglcが負の値をとらないようにする制約
  if Cglc + dCglcdt * dt < 0:
    dCglcdt = -Cglc / dt

  #乳酸代謝
  dClacdt = -qlac_algae * u * X

  #グルタミン代謝
  dCglndt = -qgln_algae * u * X - kd_gln_25 * Cgln

  #アンモニア代謝
  dCammdt = -qamm_algae * u * X + kd_gln_25 * Cgln
  # Cammが負の値をとらないようにする制約
  if Camm + dCammdt * dt < 0:   # ここで dt は時間の微小な変化量（タイムステップ）を表す
    dCammdt = -Camm / dt        # dt で割ることでタイムステップに基づいて調整


  return [dXdt, dCglcdt, dClacdt, dCglndt, dCammdt]


#藻類の比増殖速度
def u_algae(Camm):
  u = umax_algae *(Camm/(Kamm_algae+Camm))
  return u


#===============================================================================
#培地交換プロセス
#===============================================================================
#動物細胞培地
def exchange_cell(C_cell,C_algae,list_C,r):
  C = (1-r)*C_cell[-1] + r*C_algae[-1]  #培地交換後の濃度
  list_C.append(C)
  return list_C

#藻類培地
def exchange_algae(C_cell,C_algae,list_C,k_vol,r):
  C = ((k_vol - r)*C_algae[-1] + r*C_cell[-1])/k_vol  #培地交換後の濃度
  list_C.append(C)
  return list_C

## Definition of CCC model

In [ ]:
##4. CCCモデルの定義．
def CCC_model(
    umax_cell,
    qglc_cell_1,
    qglc_cell_2,
    qlac_cell_1,
    qlac_cell_2,
    qgln_cell,
    qamm_cell,
    umax_algae,
    qglc_algae,
    qlac_algae,
    qgln_algae,
    qamm_algae,
    kglc,
    k_vol,
    r,
    t_cycle
):
  # 関数の内容をここに書く

  #=============================================================================
  #CCCモデルパラメータの初期値
  #=============================================================================
  #入力パラメータの乱数生成
  #入力パラメータと標準偏差の対応をまとめる
  param_rand = {
      'list_umax_cell':   (umax_cell,   umax_cell*CV_umax_cell),
      'list_umax_algae':  (umax_algae, umax_algae*CV_umax_algae),
      'list_qglc_cell_1': (qglc_cell_1,   qglc_cell_1*CV_qglc_cell),
      'list_qglc_cell_2': (qglc_cell_2,   qglc_cell_2*CV_qglc_cell),
      'list_qlac_cell_1': (qlac_cell_1,   qlac_cell_1*CV_qlac_cell),
      'list_qlac_cell_2': (qlac_cell_2,   qlac_cell_2*CV_qlac_cell),
      'list_qgln_cell':   (qgln_cell,   qgln_cell*CV_qgln_cell),
      'list_qamm_cell':   (qamm_cell,   qamm_cell*CV_qamm_cell),
      'list_qglc_algae':  (qglc_algae, qglc_algae*CV_qglc_algae),
      'list_qlac_algae':  (qlac_algae, qlac_algae*CV_qlac_algae),
      'list_qgln_algae':  (qgln_algae, qgln_algae*CV_qgln_algae),
      'list_qamm_algae':  (qamm_algae, qamm_algae*CV_qamm_algae),
      'list_kglc':        (kglc, kglc*CV_kglc)
  }


  # 乱数生成をまとめて実行
  # 乱数生成回数
  scale = 100 #1000回
  globals().update({
      name: [random.gauss(param, sd) for i in range(scale)]
      for name, (param, sd) in param_rand.items()
  })


  # モデルパラメータの経時変化を格納するリスト
  list_names = [
      # 動物細胞培養プロセスに関連するモデルパラメータ
      'list_X_cell', 'list_u_cell', 'list_Cglc_cell', 'list_Clac_cell',
      'list_Cgln_cell', 'list_Camm_cell',
      # 藻類培養プロセスに関連するモデルパラメータ
      'list_X_algae', 'list_u_algae', 'list_Cglc_algae', 'list_Clac_algae',
      'list_Cgln_algae', 'list_Camm_algae',
      # 培地交換後の濃度（動物細胞培養プロセス）
      'list_Cglc_cell_i', 'list_Clac_cell_i', 'list_Cgln_cell_i', 'list_Camm_cell_i',
      # 培地交換後の濃度（藻類培養プロセス）
      'list_Cglc_algae_i', 'list_Clac_algae_i', 'list_Cgln_algae_i', 'list_Camm_algae_i',
      # Output
      'list_N'
  ]

  # リストをまとめて初期化
  globals().update({name: [[] for i in range(scale)] for name in list_names})


  #=============================================================================
  #CCCモデルの実行
  #=============================================================================
  #サイクル数
  cycle_num = int(t_tot / t_cycle)

  for n in range(scale):
    # flag = Trueでシミュレーション
    flag = True
    # 比増殖速度の制限
    below_umin_time = 0       # u_cell < umin_cell の状態がどれだけ続いたか追跡するための変数
    umin_cell_threshold = 10  # 10時間を閾値として設定
    # サイクルΔtにおいて比増殖速度が閾値に達した時間
    umin_time = 0

    #乱数データから入力値を設定する
    umax_cell   =  list_umax_cell[n]
    umax_algae  =  list_umax_algae[n]
    qglc_cell_1 =  list_qglc_cell_1[n]
    qglc_cell_2 =  list_qglc_cell_2[n]
    qlac_cell_1 =  list_qlac_cell_1[n]
    qlac_cell_2 =  list_qlac_cell_2[n]
    qgln_cell   =  list_qgln_cell[n]
    qamm_cell   =  list_qamm_cell[n]
    qglc_algae  =  list_qglc_algae[n]
    qlac_algae  =  list_qlac_algae[n]
    qgln_algae  =  list_qgln_algae[n]
    qamm_algae  =  list_qamm_algae[n]
    kglc        =  list_kglc[n]


    for i in range(cycle_num):
      if flag == True:
        #===========================================================================
        #動物細胞培養プロセスの初期設定
        #===========================================================================
        # 1サイクル目の初期細胞密度と培地濃度の初期値の定義．
        if i == 0:
          #初期細胞密度
          X0 = random.gauss(X0_cell,X0_cell*CV_X0_cell)
          #培地濃度の初期値
          Cglc0 = random.gauss(Cglc0_cell,Cglc0_cell*CV_Cglc0_cell)
          Clac0 = random.gauss(Clac0_cell,Clac0_cell*CV_Clac0_cell)
          Cgln0 = random.gauss(Cgln0_cell,Cgln0_cell*CV_Cgln0_cell)
          Camm0 = random.gauss(Camm0_cell,Camm0_cell*CV_Camm0_cell)
          y0 = [X0, Cglc0, Clac0, Cgln0, Camm0]
        # 2サイクル目以降の初期細胞密度と培地濃度の初期値の定義．
        else:
          #初期細胞密度(セルカウントのCVを考慮する必要がある)
          X0 = random.gauss(list_X_cell[n][-1],list_X_cell[n][-1]*CV_X_count)
          #培地濃度
          Cglc0 = list_Cglc_cell_i[n][-1]
          Clac0 = list_Clac_cell_i[n][-1]
          Cgln0 = list_Cgln_cell_i[n][-1]
          Camm0 = list_Camm_cell_i[n][-1]
          y0 = [X0, Cglc0, Clac0, Cgln0, Camm0]


        #===========================================================================
        #動物細胞培養プロセスの実行
        #===========================================================================
        #培養期間の定義
        t_span = (0, t_cycle)  # 時間範囲
        t_eval = np.linspace(*t_span, int(500*t_cycle/48))  # 評価する時間点
        # 動物細胞培養プロセスの計算
        if i == 0:  # 1サイクル目
          cell_culture = cell_culture_1
        elif list_Cglc_cell_i[n][-1] > 10:  # Cglc > 10 mmol/Lのとき
          cell_culture = cell_culture_1
        else:
          cell_culture = cell_culture_2

        sol = solve_ivp(cell_culture, t_span, y0, t_eval=t_eval)

        # 比増殖速度
        for x in range(len(sol.t)):
            # 各時点での u_cell を計算
            u = u_cell(sol.y[1][x], sol.y[2][x], sol.y[3][x], sol.y[4][x])

            # u_cell が umin_cell より小さい場合、時間を加算
            if u < umin_cell:
                below_umin_time += sol.t[x] - sol.t[x - 1]  # 時間の差分を加算
            else:
                below_umin_time = 0  # u_cell が umin_cell 以上に戻ったらリセット

            # umin_cell 以下の状態が10時間続いた場合にシミュレーションを終了
            if below_umin_time >= umin_cell_threshold:
                #print(f"Breaking simulation at t={sol.t[x]} because u_cell < umin_cell for 10 hours.")
                umin_time = x
                flag = False
                break
            # リストに比増殖速度の経時変化を格納する
            else:
              list_u_cell[n].append(u_cell(sol.y[1][x],sol.y[2][x],sol.y[3][x],sol.y[4][x]))

        # 動物細胞培養プロセスパラメータをリストに格納する
        # umin_cell 以下の状態が10時間続いた場合
        if below_umin_time >= umin_cell_threshold:
          list_X_cell[n].extend(sol.y[0][:umin_time])
          list_Cglc_cell[n].extend(sol.y[1][:umin_time])
          list_Clac_cell[n].extend(sol.y[2][:umin_time])
          list_Cgln_cell[n].extend(sol.y[3][:umin_time])
          list_Camm_cell[n].extend(sol.y[4][:umin_time])

        else:
          list_X_cell[n].extend(sol.y[0])
          list_Cglc_cell[n].extend(sol.y[1])
          list_Clac_cell[n].extend(sol.y[2])
          list_Cgln_cell[n].extend(sol.y[3])
          list_Camm_cell[n].extend(sol.y[4])


        #===========================================================================
        #動物細胞抜き出し操作
        #===========================================================================
        #回収細胞数
        list_N[n].append(list_X_cell[n][-1] * r * V)
        #抜き出し後の細胞密度
        list_X_cell[n][-1] = (1-r) * list_X_cell[n][-1]


        #===========================================================================
        #藻類培養プロセスの初期設定
        #===========================================================================
        # 1サイクル目の初期細胞密度の定義．
        if i == 0:
          #初期細胞密度
          X0 = random.gauss(X0_algae,X0_algae*CV_X0_algae)
          #培地濃度の初期値
          Cglc0 = random.gauss(Cglc0_algae,Cglc0_algae*CV_Cglc0_algae)
          Clac0 = random.gauss(Clac0_algae,Clac0_algae*CV_Clac0_algae)
          Cgln0 = random.gauss(Cgln0_algae,Cgln0_algae*CV_Cgln0_algae)
          Camm0 = random.gauss(Camm0_algae,Camm0_algae*CV_Camm0_algae)
          y0 = [X0, Cglc0, Clac0, Cgln0, Camm0]
        # 2サイクル目以降の初期細胞密度と培地濃度の初期値の定義．
        else:
          #初期細胞密度(セルカウントのCVを考慮する必要がある)
          X0 = random.gauss(list_X_algae[n][-1],list_X_algae[n][-1]*CV_X_count)
          #培地濃度
          Cglc0 = list_Cglc_algae_i[n][-1]
          Clac0 = list_Clac_algae_i[n][-1]
          Cgln0 = list_Cgln_algae_i[n][-1]
          Camm0 = list_Camm_algae_i[n][-1]
          y0 = [X0, Cglc0, Clac0, Cgln0, Camm0]


        #===========================================================================
        #5. 藻類培養プロセスの実行
        #===========================================================================
        #藻類培養プロセスの計算
        sol = solve_ivp(algae_culture, t_span, y0, t_eval=t_eval)

        # 藻類培養プロセスパラメータをリストに格納する
        if umin_time == 0:
          list_X_algae[n].extend(sol.y[0])
          list_Cglc_algae[n].extend(sol.y[1])
          list_Clac_algae[n].extend(sol.y[2])
          list_Cgln_algae[n].extend(sol.y[3])
          list_Camm_algae[n].extend(sol.y[4])

          # 比増殖速度
          for y in range(len(sol.t)):
              # 各時点での u_algae を計算
              u = u_algae(sol.y[4][y])
              list_u_algae[n].append(u_algae(sol.y[4][y]))

        # umin_cell 以下の状態が10時間続いた場合
        else:
          list_X_algae[n].extend(sol.y[0][:umin_time])
          list_Cglc_algae[n].extend(sol.y[1][:umin_time])
          list_Clac_algae[n].extend(sol.y[2][:umin_time])
          list_Cgln_algae[n].extend(sol.y[3][:umin_time])
          list_Camm_algae[n].extend(sol.y[4][:umin_time])

          # 比増殖速度
          for y in range(umin_time):
            # 各時点での u_algae を計算
            u = u_algae(sol.y[4][y])
            list_u_algae[n].append(u_algae(sol.y[4][y]))


        #===========================================================================
        #藻類栄養素抽出プロセス，藻類抜き出し操作
        #===========================================================================
        #酸加水分解反応（藻類培地にグルコースを添加）
        list_Cglc_algae[n][-1]  = list_Cglc_algae[n][-1] + r*(k_hydrolysis*kglc*list_X_algae[n][-1])/k_vol
        #抜き出し後の細胞密度
        list_X_algae[n][-1] = ((k_vol - r) * list_X_algae[n][-1])/k_vol


        #===========================================================================
        #培地交換プロセス
        #===========================================================================
        #動物細胞培地
        exchange_cell(list_Cglc_cell[n],list_Cglc_algae[n],list_Cglc_cell_i[n],r)
        exchange_cell(list_Clac_cell[n],list_Clac_algae[n],list_Clac_cell_i[n],r)
        exchange_cell(list_Cgln_cell[n],list_Cgln_algae[n],list_Cgln_cell_i[n],r)
        exchange_cell(list_Camm_cell[n],list_Camm_algae[n],list_Camm_cell_i[n],r)

        #藻類培地
        exchange_algae(list_Cglc_cell[n],list_Cglc_algae[n],list_Cglc_algae_i[n],k_vol,r)
        exchange_algae(list_Clac_cell[n],list_Clac_algae[n],list_Clac_algae_i[n],k_vol,r)
        exchange_algae(list_Cgln_cell[n],list_Cgln_algae[n],list_Cgln_algae_i[n],k_vol,r)
        exchange_algae(list_Camm_cell[n],list_Camm_algae[n],list_Camm_algae_i[n],k_vol,r)

        if flag == False:
          break


    #===========================================================================
    #培養期間Δtが480時間の約数でない場合，残り時間のプロセスを実行
    #===========================================================================
    if t_tot - t_cycle * cycle_num > 0 and flag == True:

      #===========================================================================
      #動物細胞培養プロセスの初期設定
      #===========================================================================
      #初期細胞密度(セルカウントのCVを考慮する必要がある)
      X0 = random.gauss(list_X_cell[n][-1],list_X_cell[n][-1]*CV_X_count)
      #培地濃度
      Cglc0 = list_Cglc_cell_i[n][-1]
      Clac0 = list_Clac_cell_i[n][-1]
      Cgln0 = list_Cgln_cell_i[n][-1]
      Camm0 = list_Camm_cell_i[n][-1]
      y0 = [X0, Cglc0, Clac0, Cgln0, Camm0]


      #===========================================================================
      #動物細胞培養プロセスの実行
      #===========================================================================
      #培養時間の定義
      t_span = (0, t_tot - t_cycle * cycle_num)  # 時間範囲
      t_eval = np.linspace(*t_span, int(500*(t_tot - t_cycle * cycle_num)/48))  # 評価する時間点
      # 動物細胞培養プロセスの計算
      if list_Cglc_cell_i[n][-1] > 10:  # Cglc > 10 mmol/Lのとき
        cell_culture = cell_culture_1
      else:
        cell_culture = cell_culture_2

      sol = solve_ivp(cell_culture, t_span, y0, t_eval=t_eval)

      # 比増殖速度
      for x in range(len(sol.t)):
          # 各時点での u_cell を計算
          u = u_cell(sol.y[1][x], sol.y[2][x], sol.y[3][x], sol.y[4][x])

          # u_cell が umin_cell より小さい場合、時間を加算
          if u < umin_cell:
              below_umin_time += sol.t[x] - sol.t[x - 1]  # 時間の差分を加算
          else:
              below_umin_time = 0  # u_cell が umin_cell 以上に戻ったらリセット

          # umin_cell 以下の状態が10時間続いた場合にシミュレーションを終了
          if below_umin_time >= umin_cell_threshold:
              #print(f"Breaking simulation at t={sol.t[x]} because u_cell < umin_cell for 10 hours.")
              umin_time = x
              flag = False
              break
          # リストに比増殖速度の経時変化を格納する
          else:
            list_u_cell[n].append(u_cell(sol.y[1][x],sol.y[2][x],sol.y[3][x],sol.y[4][x]))

      # 動物細胞培養プロセスパラメータをリストに格納する
      # umin_cell 以下の状態が10時間続いた場合
      if below_umin_time >= umin_cell_threshold:
        list_X_cell[n].extend(sol.y[0][:umin_time])
        list_Cglc_cell[n].extend(sol.y[1][:umin_time])
        list_Clac_cell[n].extend(sol.y[2][:umin_time])
        list_Cgln_cell[n].extend(sol.y[3][:umin_time])
        list_Camm_cell[n].extend(sol.y[4][:umin_time])

      else:
        list_X_cell[n].extend(sol.y[0])
        list_Cglc_cell[n].extend(sol.y[1])
        list_Clac_cell[n].extend(sol.y[2])
        list_Cgln_cell[n].extend(sol.y[3])
        list_Camm_cell[n].extend(sol.y[4])


      #===========================================================================
      #動物細胞抜き出し操作
      #===========================================================================
      #回収細胞数
      list_N[n].append(list_X_cell[n][-1] * r * V)
      #抜き出し後の細胞密度
      list_X_cell[n][-1] = (1-r) * list_X_cell[n][-1]


      #===========================================================================
      #藻類培養プロセスの初期設定
      #===========================================================================
      #初期細胞密度(セルカウントのCVを考慮する必要がある)
      X0 = random.gauss(list_X_algae[n][-1],list_X_algae[n][-1]*CV_X_count)
      #培地濃度
      Cglc0 = list_Cglc_algae_i[n][-1]
      Clac0 = list_Clac_algae_i[n][-1]
      Cgln0 = list_Cgln_algae_i[n][-1]
      Camm0 = list_Camm_algae_i[n][-1]
      y0 = [X0, Cglc0, Clac0, Cgln0, Camm0]


      #===========================================================================
      #藻類培養プロセスの実行
      #===========================================================================
      #藻類培養プロセスの計算
      sol = solve_ivp(algae_culture, t_span, y0, t_eval=t_eval)

      # 藻類培養プロセスパラメータをリストに格納する
      if umin_time == 0:
        list_X_algae[n].extend(sol.y[0])
        list_Cglc_algae[n].extend(sol.y[1])
        list_Clac_algae[n].extend(sol.y[2])
        list_Cgln_algae[n].extend(sol.y[3])
        list_Camm_algae[n].extend(sol.y[4])

        # 比増殖速度
        for y in range(len(sol.t)):
            # 各時点での u_algae を計算
            u = u_algae(sol.y[4][y])
            list_u_algae[n].append(u_algae(sol.y[4][y]))

      # umin_cell 以下の状態が10時間続いた場合
      else:
        list_X_algae[n].extend(sol.y[0][:umin_time])
        list_Cglc_algae[n].extend(sol.y[1][:umin_time])
        list_Clac_algae[n].extend(sol.y[2][:umin_time])
        list_Cgln_algae[n].extend(sol.y[3][:umin_time])
        list_Camm_algae[n].extend(sol.y[4][:umin_time])

        # 比増殖速度
        for y in range(umin_time):
          # 各時点での u_algae を計算
          u = u_algae(sol.y[4][y])
          list_u_algae[n].append(u_algae(sol.y[4][y]))


      #===========================================================================
      #藻類栄養素抽出プロセス
      #===========================================================================
      #酸加水分解反応（藻類培地にグルコースを添加）
      list_Cglc_algae[n][-1]  = list_Cglc_algae[n][-1] + r*(k_hydrolysis*kglc*list_X_algae[n][-1])/k_vol
      #抜き出し後の細胞密度
      list_X_algae[n][-1] = ((k_vol - r) * list_X_algae[n][-1])/k_vol


      #===========================================================================
      #培地交換プロセス
      #===========================================================================
      #動物細胞培地
      exchange_cell(list_Cglc_cell[n],list_Cglc_algae[n],list_Cglc_cell_i[n],r)
      exchange_cell(list_Clac_cell[n],list_Clac_algae[n],list_Clac_cell_i[n],r)
      exchange_cell(list_Cgln_cell[n],list_Cgln_algae[n],list_Cgln_cell_i[n],r)
      exchange_cell(list_Camm_cell[n],list_Camm_algae[n],list_Camm_cell_i[n],r)

      #藻類培地
      exchange_algae(list_Cglc_cell[n],list_Cglc_algae[n],list_Cglc_algae_i[n],k_vol,r)
      exchange_algae(list_Clac_cell[n],list_Clac_algae[n],list_Clac_algae_i[n],k_vol,r)
      exchange_algae(list_Cgln_cell[n],list_Cgln_algae[n],list_Cgln_algae_i[n],k_vol,r)
      exchange_algae(list_Camm_cell[n],list_Camm_algae[n],list_Camm_algae_i[n],k_vol,r)


  #===========================================================================
  #CCCモデルの出力
  #===========================================================================
  #最適化したいモデルの出力
  # サイクルごとの回収細胞密度の和 (Y_value), 最終サイクルの回収細胞密度の変動係数の平均値 (Y_CV)
  #1. Y_value
  Y_value = statistics.mean(sum(list_N[i]) for i in range(scale))

  #2. Y_CV
  # サイクルごとの回収細胞密度の変動係数は単調増加であるため，Y_CVが低下するとプロセスを通じてCVが低下する．
  Y_CV = statistics.stdev(list_N[i][-1] for i in range(scale))/statistics.mean(list_N[i][-1] for i in range(scale))

  return Y_value, Y_CV

---
# Output

In [ ]:
import optuna
from optuna.visualization import plot_optimization_history
import matplotlib.pyplot as plt
import csv

# 任意の定数としての lambda_value を設定
lambda_value = 1  # 必要に応じて変更 (lambda_value = 1はCVの影響が大きかった．)

# スコア計算関数
def calculate_score(Y_value, Y_CV):
    # グラフ上で凸型の曲線に沿って点が並んでいる場合（パレート解）、この曲線上を重み付き和を調整することで簡単に動ける、つまり異なる解を見つけやすい．
    return -(math.log10(Y_value) - lambda_value * Y_CV) #CV/NはCVの影響がより大きくなる．逆も然り．

# すべてのトライアル結果を格納するリスト
trial_results = []

# 目的関数の定義
def objective(trial):
    # Optunaで探索する変数の設定
    k_vol = trial.suggest_uniform('k_vol', 1, 5)
    r = trial.suggest_uniform('r', 0.1, 1)
    t_cycle = trial.suggest_int('t_cycle', 24, 120, step=12)

    # CCC_model関数の実行
    Y_value, Y_CV = CCC_model(
        umax_cell, umax_algae, qglc_cell_1, qglc_cell_2, qlac_cell_1, qlac_cell_2, qgln_cell, qamm_cell,
        qglc_algae, qlac_algae, qgln_algae, qamm_algae, kglc, k_vol, r, t_cycle
    )

    # スコアを計算
    score = calculate_score(Y_value, Y_CV)

    # 結果をリストに格納
    trial_results.append({
        'k_vol': k_vol,
        'r': r,
        't_cycle': t_cycle,
        'lambda_value': lambda_value,
        'Y_value': Y_value,
        'Y_CV': Y_CV,
        'score': score
    })

    print(f"k_vol: {k_vol}, r: {r}, t_cycle: {t_cycle}, lambda_value: {lambda_value}, Y_value: {Y_value}, Y_CV: {Y_CV}, Score: {score}")

    # スコア最小化
    return score

# OptunaのStudyの作成 (スコア最小化)
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=10)

# 収束グラフのプロット
def plot_convergence(study):
    # Optunaの収束履歴をプロット
    fig = plot_optimization_history(study)

    # グラフの装飾
    fig.update_layout(
        title="Convergence Plot",
        xaxis_title="Trial Number",
        yaxis_title="Objective Value (Score)",
        showlegend=True
    )

    # 表示
    fig.show()

# 収束グラフを表示
plot_convergence(study)

# すべてのトライアル結果をCSVに保存する
def save_all_trials(trial_results):
    csv_filename = f'all_trials_results_lambda_{lambda_value}.csv'

    # ファイルを開いてヘッダーを書き込み
    with open(csv_filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        # ヘッダー
        writer.writerow(['Trial', 'k_vol', 'r', 't_cycle', 'lambda_value', 'Y_value', 'Y_CV', 'Score'])

        # 各トライアル結果を書き込む
        for i, result in enumerate(trial_results):
            writer.writerow([i + 1, result['k_vol'], result['r'], result['t_cycle'],
                             result['lambda_value'], result['Y_value'], result['Y_CV'], result['score']])

        print(f"All trial results saved to {csv_filename}")

# 最適な初期値を取得（最適トライアルは1つ）
def get_best_initial_params(study):
    best_trial = study.best_trial  # 最適トライアルを取得

    best_params = {
        'k_vol': best_trial.params['k_vol'],
        'r': best_trial.params['r'],
        't_cycle': best_trial.params['t_cycle'],
        'lambda_value': lambda_value,
    }

    return best_params

# 最適トライアルの結果をリストから取得
def get_best_trial_result(trial_results, best_params):
    for result in trial_results:
        if (result['k_vol'] == best_params['k_vol'] and
            result['r'] == best_params['r'] and
            result['t_cycle'] == best_params['t_cycle'] and
            result['lambda_value'] == best_params['lambda_value']):
            return result['Y_value'], result['Y_CV']

# 最適トライアルの結果をCSVに保存
def save_best_trial(best_params, E_Y, V_Y):
    csv_filename = f'best_trial_results_lambda_{lambda_value}.csv'

    # ファイルを開いてヘッダーを書き込み
    with open(csv_filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        # ヘッダー
        writer.writerow(['k_vol', 'r', 't_cycle', 'lambda_value', 'E_Y', 'V_Y'])

        # 最適なパラメータと期待値・分散を書き込む
        writer.writerow([
            best_params['k_vol'], best_params['r'], best_params['t_cycle'],
            best_params['lambda_value'], E_Y, V_Y
        ])

        print(f"Best trial results saved to {csv_filename}")

# すべてのトライアル結果をCSVに保存
save_all_trials(trial_results)

# 最適な初期値を取得
best_params = get_best_initial_params(study)

# 最適トライアルの期待値と分散をリストから取得
E_Y, V_Y = get_best_trial_result(trial_results, best_params)

# 最適トライアルの結果をCSVに保存
save_best_trial(best_params, E_Y, V_Y)

[I 2024-11-22 02:01:06,955] A new study created in memory with name: no-name-f1d2d0f2-001b-4b24-a296-66fdc063704f
<ipython-input-6-347d194cf187>:20: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  k_vol = trial.suggest_uniform('k_vol', 1, 5)
<ipython-input-6-347d194cf187>:21: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  r = trial.suggest_uniform('r', 0.1, 1)
[I 2024-11-22 02:01:18,282] Trial 0 finished with value: -7.612997478723176 and parameters: {'k_vol': 3.983273575895545, 'r': 0.1275866838369859, 't_cycle': 24}. Best is trial 0 with value: -7.612997478723176.


k_vol: 3.983273575895545, r: 0.1275866838369859, t_cycle: 24, lambda_value: 1, Y_value: 48546050.356090575, Y_CV: 0.0731564233204134, Score: -7.612997478723176


[I 2024-11-22 02:01:32,456] Trial 1 finished with value: -7.593639781114953 and parameters: {'k_vol': 4.544330225330203, 'r': 0.7718252867350867, 't_cycle': 36}. Best is trial 0 with value: -7.612997478723176.


k_vol: 4.544330225330203, r: 0.7718252867350867, t_cycle: 36, lambda_value: 1, Y_value: 130296262.63821875, Y_CV: 0.5212921776607432, Score: -7.593639781114953


[I 2024-11-22 02:01:43,508] Trial 2 finished with value: -8.348013420838127 and parameters: {'k_vol': 2.6941612574548115, 'r': 0.4717260083949706, 't_cycle': 36}. Best is trial 2 with value: -8.348013420838127.


...

[I 2024-11-22 02:02:11,682] Trial 8 finished with value: -7.976245887898573 and parameters: {'k_vol': 1.3825786949601677, 'r': 0.9323514508008045, 't_cycle': 60}. Best is trial 2 with value: -8.348013420838127.


k_vol: 1.3825786949601677, r: 0.9323514508008045, t_cycle: 60, lambda_value: 1, Y_value: 141188331.57543123, Y_CV: 0.17355291829455932, Score: -7.976245887898573


[I 2024-11-22 02:02:22,521] Trial 9 finished with value: -7.469868002867009 and parameters: {'k_vol': 4.760814516576887, 'r': 0.87282458749458, 't_cycle': 36}. Best is trial 2 with value: -8.348013420838127.


k_vol: 4.760814516576887, r: 0.87282458749458, t_cycle: 36, lambda_value: 1, Y_value: 100721623.8232555, Y_CV: 0.5332547159384623, Score: -7.469868002867009


All trial results saved to all_trials_results_lambda_1.csv
Best trial results saved to best_trial_results_lambda_1.csv
